## Convolutional Neural Networks

Pada lab sebelumnya, telah dibuat dan ditrain simpel model untuk klasifikasi citra ASL. Model meliki akurasi yang tinggi pada training data, tetapi untuk validation data tidak perform sebaik di training data. Pada lab ini akan dibuat model cnn yang sangat bagus untuk klasifikasi citra

## Objectives




*   Prepare data untuk CNN
*   Buat CNN model
*   Train CNN model dan mengamati performanya





## 1. Prepare Data

In [29]:
import tensorflow.keras as keras
import pandas as pd

# load data dari CSV files
train_df = pd.read_csv("/content/sign_mnist_train.csv")
valid_df = pd.read_csv("/content/sign_mnist_valid.csv")

# extract nilai target/label
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# ambil nilai image data
x_train = train_df.values
x_valid = valid_df.values

# ubah nilai target/label ke categorical
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# normalize image data
x_train = x_train / 255
x_valid = x_valid / 255

Reshaping images data untuk CNN model

Pada lab sebelumnya, image data dalam format 1d array 784 pixels. Pada format ini, kita tidak memiliki semua informasi mengenai pixel-pixel mana yang berdekatan satu sama lain. Karena ini, kita tidak dapat melakukan convolution untuk detect features penting

In [30]:
print(x_train.shape)
print(x_valid.shape)

(27455, 784)
(7172, 784)


Reshape dataset kedalam format 28x28 pixel. Untuk layer convolutional pertama dari model, perlu tidak hanya height dan width dari citra, tetapi juga jumlah color channels. Karena citra merupakan grayscale, maka hanya memiliki 1 color channel

Oleh karena itu, perlu konversi dari shape (27455, 784) menjadi (27455, 28, 28, 1).

In [31]:
# nilai -1 pada argumen pertama jumlah citra tetap sama dengan citra awal
x_train = x_train.reshape(-1, 28, 28, 1)
x_valid = x_valid.reshape(-1, 28, 28, 1)

In [32]:
x_train.shape

(27455, 28, 28, 1)

In [33]:
x_valid.shape

(7172, 28, 28, 1)

In [34]:
x_train[0].shape

(28, 28, 1)

In [35]:
y_train[0].shape

(24,)

## 2. Create a CNN Model

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))

Conv2D merupakan 2D convolutional layers. Convolution merupakan perkalian antara antara filter/kernel dengan citra. Convolution dilakukan untuk detect feature2 yang penting untuk klasifikasi. Earlier convolution akan detect simpel feature seperti garis, later convolution akan detect feature yang lebih kompleks.

model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu",
                 input_shape=(28, 28, 1)))

75 merupakan jumlah filter yang akan dipelajari. (3,3) merupakan ukuran filter, strides merujuk ke seberaja jauh cel filter akan digeser ketika proses convolution. padding="same" akan memastikan filter diterapkan ke semua nilai dari input, output image dari convolution akan memiliki size yang sama dengan input image (untuk strides=1)

Batch Normalization merupakan layer yang menormalize inputnya, menerapkan transformasi yang mempertahankan mean dari output dekat ke 0 dan standard deviation output dekat ke 1. Ini berguna untuk menstabilkan proses learning dan secara dramatis mengurangi jumlah training epochs yang diperlukan untuk train model.

MaxPool2D merupakan Max Pooling layer yang berguna untuk mengurangi resolusi dari citra. Hal ini dilakukan dengan menaruh window pada citra kemudian mengambil nilai maksimalnya.

model.add(MaxPool2D((2, 2), strides=2, padding="same"))

(2,2) merupakan ukuran window

Dropout merupakan teknik yang digunakan untuk mencegah overfitting. Dropout akan memilih subset neuoron secara random kemudian mematikkan neuron tersebut, sehingga mereka tidak bisa learn (berpartisipasi dalam forward dan backward propagation). Hal ini dilakukan untuk memastikan model robust, tidak overreliance (sangat bergantung) pada input atau neuron tertentu.

model.add(Dropout(0.2))

0.2 merupakan rate dropout. 0 berarti tidak ada neuron yang dimatikan, 1 semua neuron dimatikan

Flatten takes output dari satu layer (multidimensional), kemudian flatten/unroll kedalam 1 dimensional array. Hasilnya disebut feature vector dan akan terhubung dengan final cllasification layer

Dense

dense layer pertama (512 units) takes feature vector sebagai input dan akan mempelajari feature mana yang akan berkontribusi untuk particular classification. dense layer kedua (24 units) merupakan final classification layer yang menghasilkan prediksi dari model dalam bentuk probabilitas (karena softmax activation func.)

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 28, 28, 75)        750       
                                                                 
 batch_normalization_9 (Batc  (None, 28, 28, 75)       300       
 hNormalization)                                                 
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 14, 14, 75)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 14, 14, 50)        33800     
                                                                 
 dropout_6 (Dropout)         (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_10 (Bat  (None, 14, 14, 50)      

Dari model summary diatas, ada 300 non-trainable paramas karena terdapat beberapa neuron yang kita matikan

In [38]:
# compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

## 3. Train the Model

In [39]:
model.fit(x_train, y_train, epochs=20, verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/20
858/858 [==============================] - 116s 131ms/step - loss: 0.3035 - accuracy: 0.9073 - val_loss: 0.2254 - val_accuracy: 0.9140
Epoch 2/20
858/858 [==============================] - 110s 128ms/step - loss: 0.0186 - accuracy: 0.9935 - val_loss: 0.5520 - val_accuracy: 0.8660
Epoch 3/20
858/858 [==============================] - 109s 127ms/step - loss: 0.0110 - accuracy: 0.9962 - val_loss: 0.2878 - val_accuracy: 0.9381
Epoch 4/20
858/858 [==============================] - 112s 130ms/step - loss: 0.0071 - accuracy: 0.9980 - val_loss: 0.2381 - val_accuracy: 0.9385
Epoch 5/20
858/858 [==============================] - 112s 130ms/step - loss: 0.0071 - accuracy: 0.9979 - val_loss: 0.2643 - val_accuracy: 0.9494
Epoch 6/20
858/858 [==============================] - 111s 129ms/step - loss: 0.0043 - accuracy: 0.9986 - val_loss: 0.2316 - val_accuracy: 0.9516
Epoch 7/20
858/858 [==============================] - 111s 130ms/step - loss: 0.0040 - accuracy: 0.9989 - val_loss: 0.2097 -

Terlihat model sudah meningkat secara signifikan. Trining accuracy sangat tinggi, dan validation accuracy telah meningkat juga. Akan tetapi validation accuracy masih melompat-lompat yang menandandakan model masih belum bisa melakukan generalisasi secara sempurna. Lab selanjutnya akan mencoba mengatasi permasalahan ini